<a href="https://colab.research.google.com/github/bdafahim/AI_Assignment/blob/main/Text_Suggstion_Using_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus

In [2]:
pathForWikiDataset = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
wikiSentences = WikiCorpus(pathForWikiDataset).get_texts()
print(wikiSentences)

<generator object WikiCorpus.get_texts at 0x7fa36f51bfc0>


In [3]:
nltk.download('punkt')
nltk.download('reuters')
reutersSentences  = reuters.sents()
print(reutersSentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [5]:
def calculateProbabilities(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    total_Word_Count = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord]/=total_Word_Count

In [6]:
def calculateSigleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word]/=wordCount

In [7]:
def convertToLower(pa):
  if type(pa)==str:
    return pa.lower()
  return pa

In [8]:
sentenceModel4 = defaultdict(lambda: set())
sentenceModel5 = defaultdict(lambda: set())

def calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    #print(sentence)
    for word in sentence:
      wordCount+=1
      sentenceModel1[word]+=1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      # print(previousWord1)
      # print(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1]+=1
      sentenceModel3[nextWord][previousWord2]+=1
      sentenceModel4[previousWord1].add(nextWord)
      sentenceModel5[previousWord2].add(nextWord)

  return wordCount

In [9]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0))
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))

In [10]:
wikiWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,wikiSentences)
print(wikiWordCount)

452944


In [11]:
reutersWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,reutersSentences)
print(reutersWordCount)

1720917


In [13]:
calculateProbabilities(sentenceModel2)

In [14]:
calculateProbabilities(sentenceModel3)

In [15]:
total_word = wikiWordCount + reutersWordCount
calculateSigleWordProbability(sentenceModel1,total_word)

In [ ]:
#@title Naive Bayes


In [16]:
maxProbabilityWords = []
def makeWordSuggestionByTrigram(previousWord2,previousWord1):
  for nextWord in sentenceModel4[previousWord1] & sentenceModel5[previousWord2]:
    naiveBiasTrigramWeight = sentenceModel1[nextWord]*sentenceModel2[nextWord][previousWord1]*sentenceModel3[nextWord][previousWord2]
    maxProbabilityWords.append((nextWord,naiveBiasTrigramWeight))

In [17]:
makeWordSuggestionByTrigram('my','name')
maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
print(*maxProbabilityWords[:10])

('is', 2.6849293640882563e-08) ('to', 1.1753701280369476e-09) (',', 5.594384896959364e-10) ('in', 4.6278217257886463e-10) ('and', 4.5063509855295003e-10) ('for', 3.7387907837491414e-10) ('would', 3.538344188456149e-10) ('will', 2.873504539086109e-10) ('that', 2.0690529880633974e-10) ('or', 1.711572663102721e-10)


In [18]:
while(True):
    text = input("Enter your line: ")
    if text == "stop":
        print("Ending The Program.....")
        break
    
    else:
        try:
            maxProbabilityWords = []
            text = text.split(" ")
            makeWordSuggestionByTrigram(text[0],text[1])
            maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
            print(*maxProbabilityWords[:10])
            
        except:
            continue

Enter your line: I am

Enter your line: you are
('not', 1.3601059039706865e-07) ('re', 7.599793212130158e-08) ('going', 6.093710092694297e-08) ('still', 4.9851741930418823e-08) ('trying', 4.696590473574191e-08) ('certainly', 4.6268700091520467e-08) ('normally', 3.930867495990026e-08) ('considered', 2.8569585597962102e-08) ('bound', 2.8163939550835283e-08) ('already', 2.3833544823549687e-08)
Enter your line: he is
('not', 1.1415344990518883e-05) ('the', 2.8669512353900393e-06) ('no', 2.0306414186860617e-06) ('that', 1.5753769451114705e-06) ('also', 1.1663932699492465e-06) ('a', 1.0837436335618735e-06) ('confident', 9.282990400880168e-07) ('considering', 7.529942171577492e-07) ('an', 7.262667551027575e-07) ('concerned', 5.642801755340686e-07)
Enter your line: she is
('nonviolent', 4.600110126636432e-07) ('not', 2.8951961932475424e-07) ('concerned', 2.2571207021362745e-07) ('asleep', 1.672767318776884e-07) ('expected', 1.5764486617874928e-07) ('seated', 1.5333700422121439e-07) ('the', 1.3